
# Reconhecimento de Entidades Nomeadas (NER)


Modelos de Reconhecimento de Entidades Nomeadas visam extrair do texto informações categorizadas como:
- Pessoa (PER): Nome de pessoas.
- Organização (ORG): Empresas e instituições.
- Localização (LOC): Cidades, países, regiões geográficas.
- Miscellaneous (MISC): Outros, o que não for das categorias acima mas ainda assim for importante, como evento ou produtos.
<br>A ideia do NER é classificar cada token dentro do texto entre uma dessas opções.
<br><br>A forma de classificação mais comum é a Inside-Outside-Beginning (IOB):
- B- (Beginning): Marca o início de uma entidade.
- I- (Inside): Marca o token de dentro da entidade, contínuo ao anterior.
- O (Outside): Marca um token que não faz parte de nenhuma entidade nomeada.
<br>Nesse formato, o modelo tem todas as informações para distinguir de forma clara quando uma entidade nomeada começa e quando termina.

#### Usando modelos pré-treinados

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('maxent_ne_chunker_tab')

from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /home/carloslessa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/carloslessa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/carloslessa/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/carloslessa/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /home/carloslessa/nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!


In [2]:
texto = "Bem vindos a aula de Reconhecimento de Entidades Nomeadas da FIAP ministrada pelo professor Dheny Fernandes"
tokens = word_tokenize(texto) # separa o texto em tokens
taggeado = pos_tag(tokens) # realiza o processo de POS-tagging
entidades = ne_chunk(taggeado) # detecta as entidades
print(entidades)

(S
  (GPE Bem/NNP)
  vindos/VBZ
  a/DT
  aula/NN
  de/FW
  (ORGANIZATION Reconhecimento/NNP)
  de/FW
  (PERSON Entidades/NNP Nomeadas/NNP)
  da/NN
  (ORGANIZATION FIAP/NNP)
  ministrada/NNP
  pelo/NN
  professor/NN
  (PERSON Dheny/NNP Fernandes/NNP))


##### NLTK
Modelo pré-treinado no idioma inglês.
Podemos ver que ele se perde bastante no exemplo usado, classificando o "Bem" como GPE (localização), Reconhecimento como organização, mas também acertar em outras, como FIAP como Organização e Dheny Fernandes como pessoa.

Ainda conseguimos usar mesmo que com um erro maior alguns modelos pré-treinados em outros idiomas, desde que eles possuam padrões em comum. Por exemplo, as línguas da família Indo-Européia como espanhol, inglês, português, russo, alemão, francês e outros, possuem padrões que podem fazer com que um modelo pré-treinado em uma, possa ser usado em outra. Já se fossemos testar em textos japoneses, esses modelos teriam uma performance bem mais baixa, visto que são idiomas bem diferentes.

Se testaremos o mesmo texto em inglês, vemos que o resultado melhora bastante, mesmo que ainda fazendo um pequeno erro.

In [3]:
text = "Welcome to the Named Entity Recognition class from FIAP lectured by professor Dheny Fernandes"
tokens = word_tokenize(text)
taggeado = pos_tag(tokens)
entidades = ne_chunk(taggeado)
print(entidades)

(S
  Welcome/VB
  to/TO
  the/DT
  (ORGANIZATION Named/NNP Entity/NNP)
  Recognition/NNP
  class/NN
  from/IN
  (ORGANIZATION FIAP/NNP)
  lectured/VBN
  by/IN
  professor/NN
  (PERSON Dheny/NNP Fernandes/NNP))


##### Spacy
A biblioteca spacy possui um modelo NER pré-treinado no idioma português, vamos ver como ele se sai:

In [4]:
import spacy

nlp = spacy.load('pt_core_news_sm')
doc = nlp(texto)
for ent in doc.ents:
    print(ent.text, ent.label_)

Reconhecimento de Entidades Nomeadas da FIAP LOC
Dheny Fernandes PER


##### Fine-tuned BERT
O modelo abaixo foi pré-treinado na tarefa de reconhecimento de entidades nomeadas usando o BERTimbau, que é pré-treinado na língua portuguesa. O modelo foi pré-treinado usando artigos da Globo News. Mais informações sobre o modelo estão disponíveis na [documentação](https://huggingface.co/monilouise/ner_news_portuguese).

Abaixo podemos ver que FIAP não foi identificada como uma entidade nomeada, mas Dheny foi identifado, mas nao da maneira esperada

In [5]:
from transformers import BertForTokenClassification, DistilBertTokenizerFast, pipeline
model = BertForTokenClassification.from_pretrained(
    'monilouise/ner_pt_br', cache_dir="../models/"
)
tokenizer = DistilBertTokenizerFast.from_pretrained(
    'neuralmind/bert-base-portuguese-cased',
    model_max_length=512,
    do_lower_case=False,
    cache_dir="../models/"
)
ner = pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True, device=-1)
entidades = ner(texto)
for ent in entidades:
    print(f"{ent['word']}: {ent['entity_group']} ({ent['score']:.2f})")

/home/carloslessa/FCD/POSTECH/modulo4/02-NLP/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at monilouise/ner_pt_br were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is 

D: PESSOA (0.95)
##hen: PESSOA (0.78)
##y: PESSOA (0.44)


Vamos testar mais um exemplo:

In [6]:
entidades = ner("João é funcionário do banco Itau")
for ent in entidades:
    print(f"{ent['word']}: {ent['entity_group']} ({ent['score']:.2f})")

João: PESSOA (0.74)
banco: ORG (0.53)
Ita: ORG (0.90)
##u: ORG (0.73)


Agora que já usamos modelos prontos, vamos ver como treinar o nosso próprio?

#### Treinando um modelo multi linguagem

Vamos treinar um modelo multi linguagem para Reconhecimento de Entidade Nomeada. Para isso, vamos usar um pedaço do dataset “XTREME” (Cross-lingual TRansfer Evaluation of Multilingual Encoders), chamado de WikiANN ou PAN-X.

In [7]:
# !pip install datasets

In [8]:
from collections import defaultdict, Counter

import torch
import pandas as pd
import numpy as np
from datasets import DatasetDict, load_dataset, get_dataset_config_names
from transformers import (
    XLMRobertaForTokenClassification,
    AutoTokenizer,
    AutoConfig,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
from seqeval.metrics import f1_score

Esse dataset possui 183 subsets (idiomas), com um padrão de PAN-x. e a sigla do idioma.

In [9]:
xtreme_subsets = get_dataset_config_names("xtreme") 
print(f"XTREME possui {len(xtreme_subsets)} configurações.") 
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
print(panx_subsets[:3])

XTREME possui 183 configurações.
['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']


Vamos pegar os seguintes idiomas: inglês, português e francês. Além disso, vamos fazer uma redução do dataset por proporções para termos uma visão mais realista. Datasets desbalanceados são muito comuns em problemas na vida real.

In [10]:
langs = ['pt', 'en', 'fr']
prop = [0.13, 0.72, 0.15]
panx_ch = defaultdict(DatasetDict)
for lang, prop in zip(langs, prop):
	ds = load_dataset('xtreme', name=f'PAN-X.{lang}')
	for split in ds:
		panx_ch[lang][split] = ds[split].shuffle(seed=0).select(
            range(int(prop * ds[split].num_rows))
        )
pd.DataFrame(
	{
		lang: [panx_ch[lang]['train'].num_rows]
		for lang in langs
	}, index=['Número de observações de treino']
)


,pt,en,fr
Número de observações de treino,2600,14400,3000


Primeiramente vamos treinar nosso modelo na língua inglesa e testarmos ele na língua inglesa, portuguesa e francesa.
Vamos inspecionar um exemplo no dataset inglês.

In [11]:
exemplo = panx_ch['en']['train'][0]
for key, value in exemplo.items():
    print(f'{key}: {value}')

tokens: ["'", "''", 'Toronto', 'Lynx', "''", "'"]
ner_tags: [0, 0, 3, 4, 0, 0]
langs: ['en', 'en', 'en', 'en', 'en', 'en']


Podemos ver que a coluna ner_tags corresponde a um código mapeado, então vamos transformá-lo em uma nova coluna mais familiar, com as tags LOC, PER e ORG. Podemos fazer isso através do atributo features.

In [12]:
for key, value in panx_ch['en']['train'].features.items():
	print(f'{key}: {value}')

tokens: List(Value('string'))
ner_tags: List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))
langs: List(Value('string'))


Agora vamos criar uma função que cria a coluna ner_tags_str usando a feature ner_tags que possui uma classe ClassLabel que mapeia o id com a tag.

In [13]:
tags = panx_ch['en']['train'].features['ner_tags'].feature
def create_tag_names_column(batch):
	return {'ner_tags_str': [tags.int2str(idx) for idx in batch['ner_tags']]}
panx_en = panx_ch['en'].map(create_tag_names_column)
exemplo = panx_en['train'][0]
pd.DataFrame([exemplo['tokens'], exemplo['ner_tags_str']], ['tokens', 'tags'])


,0,1,2,3,4,5
tokens,','',Toronto,Lynx,'','
tags,O,O,B-ORG,I-ORG,O,O


Acima já vemos alguns desafios que enfrentamos lidando com idiomas: Toronto é uma cidade, que seria classificado como local, mas Toronto Lynx é um time de futebol, então é classificado como organização. Outro desafio importante é a criação das tags, podemos ver que é bem trabalhoso e complexo de taggear todas as sentenças de um dataset inteiro para treinarmos o modelo.

Agora faremos uma Análise Exploratória para entender a quantidade de tags que temos em cada dataset e ver se estão homogênias.

In [14]:
split2freqs = defaultdict(Counter)
for split, dataset in panx_en.items():
	for row in dataset['ner_tags_str']:
		for tag in row:
			if tag.startswith('B'):
				tag_type = tag.split('-')[1]
				split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient='index')


,ORG,PER,LOC
train,6715,6542,6767
validation,3364,3276,3458
test,3385,3339,3357


Agora que vimos o dataset, vamos para o modelo.

Nesse exemplo, usaremos o modelo [XLM-RoBERTa](https://huggingface.co/docs/transformers/model_doc/xlm-roberta). O modelo RoBERTa modifica o pré-treinamento do BERT, melhorando-o ao treinar com mais dados por mais tempo, além de não realizar a tarefa de Next Sentence Prediction. O XLM-RoBERTa, por sua vez, estende o RoBERTa para um pré-treinamento multi-língua. Outra diferença para o BERT é a tokenização: Enquanto o BERT tokeniza usando WordPiece, o XLM-RoBERTa utiliza o SentencePiece. Essa mudança de tokenização é especialmente importante para lidar com línguas em que a separação por espaço não faz sentido, como na língua japonesa, sendo mais agnóstica.

In [15]:
CUDA_VISIBLE_DEVICES="" # Força o uso da CPU

model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="../models/")
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
config = AutoConfig.from_pretrained(
	model_name,
	num_labels=tags.num_classes,
	id2label=index2tag,
	label2id=tag2index,
    cache_dir="../models/"
)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
model = XLMRobertaForTokenClassification.from_pretrained(
    model_name, config=config, cache_dir="../models/"
).to(device)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Antes de fazer o fine-tuning do modelo, precisamos tokenizar nosso dataset e alinhar e limpar nossos labels. 

Não queremos que tokens de caracter especial, como o `<s>` e o `<\s>`,que indicam início e fim de sentença, sejam classificados, e também queremos ignorar a classificação das subpalavras; somente a primeira deve ser classificada, as demais serão tratadas posteriormente quando juntarmos os tokens em palavras. Para isso, vamos usar os word_ids. Vamos ver um exemplo abaixo.


In [16]:
tokenized_input = tokenizer(exemplo['tokens'], is_split_into_words=True) 
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"]) 
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens, word_ids], index=["Tokens", "Word Ids"]) 

,0,1,2,3,4,5,6,7,8
Tokens,<s>,▁',▁'',▁Toronto,▁Lyn,x,▁'',▁',</s>
Word Ids,None,0,1,2,3,3,4,5,None


Podemos ver acima que os caracteres especiais estão com Ids None. Além disso, as subpalavras possuem o mesmo id da anterior: A palavra Lynx foi dividida em ▁Lyn e x por exemplo, ambas com o id 3. Com isso, podemos tratar os labels para quando forem None ou iguais ao anterior, alocamos o label id -100. Porque -100? Porque a classe torch.nn.CrossEntropyLoss possui um atributo chamado ignore_index com valor -100. Dessa forma, os tokens associados com label -100 serão ignorados durante o treinamento.

In [17]:
def tokenize_and_align_labels(examples):
	tokenized_inputs = tokenizer(
		examples['tokens'], truncation=True, is_split_into_words=True
	)
	labels = []
	for idx, label in enumerate(examples['ner_tags']):
		word_ids = tokenized_inputs.word_ids(batch_index=idx)
		previous_word_idx = None
		labels_ids = []
		for word_idx in word_ids:
			if word_idx is None or word_idx == previous_word_idx:
				labels_ids.append(-100)
			else:
				labels_ids.append(label[word_idx])
			previous_word_idx = word_idx
		labels.append(labels_ids)
	tokenized_inputs['labels'] = labels
	return tokenized_inputs
def encode_panx_dataset(corpus):
	return corpus.map(
		tokenize_and_align_labels,
		batched=True,
		remove_columns=['langs', 'ner_tags', 'tokens']
	)
panx_en_encoded = encode_panx_dataset(panx_ch['en'])


A avaliação de um modelo NER é similar ao de modelos de classificação, usando métricas como acuracidade, precision, recall, f1-score. A diferença é que todas as palavras de uma entidade precisam ser previstas corretamente para uma previsão contar como correta. Para isso, usaremos a biblioteca seqeval, que faz exatamente isso. Veja um exemplo abaixo.

In [18]:
!pip install seqeval

In [19]:
from seqeval.metrics import classification_report
y_true = [
    ['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'],
    ['B-PER', 'I-PER', 'O']
]
y_pred = [
    ['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'],
    ['B-PER', 'I-PER', 'O']
]
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

        MISC       0.00      0.00      0.00         1
         PER       1.00      1.00      1.00         1

   micro avg       0.50      0.50      0.50         2
   macro avg       0.50      0.50      0.50         2
weighted avg       0.50      0.50      0.50         2



Para usar essa biblioteca, precisaremos formatar as previsões e os labels para formato de lista de listas, conforme visto acima, onde cada lista corresponde a um registro do dataset.

In [20]:
def align_predictions(predictions, label_ids):
	preds = np.argmax(predictions, axis=2)
	batch_size, seq_len = preds.shape
	labels_list, preds_list = [], []
	for batch_idx in range(batch_size):
		example_label, example_preds = [], []
		for seq_idx in range(seq_len):
			if label_ids[batch_idx, seq_idx] != -100:
				example_label.append(
                    index2tag[label_ids[batch_idx][seq_idx]]
                )
				example_preds.append(
                    index2tag[preds[batch_idx][seq_idx]]
                )
		labels_list.append(example_label)
		preds_list.append(example_preds)
	return preds_list, labels_list


Agora temos tudo que precisamos para fazer o fine-tuning do XLM-RoBERTa. Primeiramente, faremos somente no subset do idioma inglês e testaremos o modelo nos demais idiomas, português e francês.

In [21]:
num_epochs = 3
batch_size = 24
logging_steps = len(panx_en_encoded['train']) // batch_size
finetuned_model_name = f'../models/{model_name}-finetuned-panx-en'
training_args = TrainingArguments(
    output_dir=finetuned_model_name,
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy='epoch',
    save_steps=1e6,
    weight_decay=0.01,
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=False,
	use_cpu=True,
)

def compute_metrics(eval_pred):
	y_pred, y_true = align_predictions(
        eval_pred.predictions, eval_pred.label_ids
    )
	return {'f1': f1_score(y_true, y_pred)}

Por fim, precisamos aplicar o método pad para deixar todas as sequências do mesmo tamanho em uma batch (lote). Para isso, usaremos o data collator. Precisamos aplicá-lo tanto no texto como nos labels, porque agora temos labels de tamanhos diferentes, são labels sequenciais. O pad seguirá a mesma lógica implementada, transformando os tokens pad para ids de -100 para serem ignorados durante o treinamento.

In [22]:
data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=panx_en_encoded['train'],
    eval_dataset=panx_en_encoded['validation'],
    tokenizer=tokenizer
)
trainer.train()

/tmp/ipykernel_3718146/3045119286.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1
1,0.442600,0.299727,0.774103
2,0.248400,0.268410,0.790067
3,0.174100,0.263584,0.816475


/home/carloslessa/FCD/POSTECH/modulo4/02-NLP/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 Quadro P620 which is of cuda capability 6.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/home/carloslessa/FCD/POSTECH/modulo4/02-NLP/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/home/carloslessa/FCD/POSTECH/modulo4/02-NLP/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:326: UserWarning: 
Quadro P620 with CUDA capability sm_61 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the Quadro P620 GPU with PyTorch

TrainOutput(global_step=1800, training_loss=0.2883507580227322, metrics={'train_runtime': 5203.6627, 'train_samples_per_second': 8.302, 'train_steps_per_second': 0.346, 'total_flos': 840013140205008.0, 'train_loss': 0.2883507580227322, 'epoch': 3.0})

O treinamento pode demorar bastante, tenha paciência. Acima podemos ver que os resultados foram muito bons! Com F1 de cerca de 81%! Após, já teremos nosso modelo treinado, agora é só formatar o resultado final.

In [23]:
def tag_text(text, tags, model, tokenizer):
	tokens = tokenizer(text).tokens()
	input_ids = tokenizer(text, return_tensors='pt').input_ids.to(device)
	outputs = model(input_ids)[0]
	predictions = torch.argmax(outputs, dim=2)
	preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
	return pd.Dataframe([tokens, preds], index=['Tokens', 'Tags'])

def get_f1_score(trainer, dataset):
    return trainer.predict(dataset).metrics['test_f1']

def evaluate_lang_performance(lang, trainer):
	panx_ds = encode_panx_dataset(panx_ch[lang])
	return get_f1_score(trainer, panx_ds['test'])

O nosso modelo até então foi treinado somente em texto do idioma inglês, será que ele já performa bem nos idiomas português e francês?

In [24]:
f1_scores = defaultdict(dict)
for lang in langs:
	f1_scores['en'][lang] = evaluate_lang_performance(lang, trainer)
f1_scores

Map: 100%|██████████| 1300/1300 [00:00<00:00, 16863.77 examples/s]


Map: 100%|██████████| 7200/7200 [00:00<00:00, 14685.34 examples/s]


Map: 100%|██████████| 1500/1500 [00:00<00:00, 15099.81 examples/s]


defaultdict(dict,
            {'en': {'pt': 0.7487598482637876,
              'en': 0.8159874145813873,
              'fr': 0.7477233571252768}})

Incrível! Mesmo o treinamento sendo no idioma inglês, já conseguimos usar esse modelo nos outros idiomas com uma boa performance, com F1 de cerca de 75%.
<br>Mas ainda podemos melhorar, podemos treinar um modelo em todos os idiomas de uma vez.


In [ ]:
from datasets import concatenate_datasets
def concatenate_splits(corpus):
	multi_corpus = DatasetDict()
	for split in corpus[0].keys():
		multi_corpus[split] = concatenate_datasets(
			[data[split] for data in corpus]
		).shuffle(seed=42)
	return multi_corpus

panx_all_encoded = [panx_en_encoded]
for lang in ['pt', 'fr']:
	panx_encoded = encode_panx_dataset(panx_ch[lang])
	panx_all_encoded.append(panx_encoded)
corpus = concatenate_splits(panx_all_encoded)
training_args.output_dir = f'../models/{model_name}-finetuned-panx-all'
training_args.logging_steps = len(corpus['train']) // batch_size
trainer = Trainer(model,args=training_args, data_collator=data_collator, compute_metrics=compute_metrics, train_dataset=corpus['train'], eval_dataset=corpus['validation'], tokenizer=tokenizer)
trainer.train()

Map: 100%|██████████| 2600/2600 [00:00<00:00, 17932.39 examples/s]
/tmp/ipykernel_3718146/833909595.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model,args=training_args, data_collator=data_collator, compute_metrics=compute_metrics, train_dataset=corpus['train'], eval_dataset=corpus['validation'], tokenizer=tokenizer)


Epoch,Training Loss,Validation Loss,F1
1,0.246400,0.266542,0.813503


Que resultado! 83% de F1 score no total! Agora vamos testar o modelo multi linguagem nos nossos datasets e ver como ficou.

In [ ]:
for idx, lang in enumerate(langs):
	f1_scores['all'][lang] = get_f1_score(
		trainer, panx_all_encoded[idx]['test']
	)
scores_data = {
	'en': f1_scores['en'],
	'all': f1_scores['all'],
}
f1_scores_df = pd.DataFrame(scores_data).T.round(4).rename_axis(
	index='Fine-tuned on', columns= 'Evaluated on'
)
f1_scores_df

Veja que usar outros idiomas no treinamento aumentou o resultado testando somente no dataset em inglês também, indo de 82% para 87% em F1! Isso porque ele aprendeu outros padrões com os outros idiomas que puderam ser aproveitados. No entanto, isso é verdade para aqueles idiomas que são similares, conforme comentamos anteriormente, todos os idiomas testados pertencem à família Indo-Européia, que compartilham padrões entre si. Esses padrões fazem com que o modelo performe de forma quase equivalente nesses idiomas, podemos ver que o F1 em todos os três idiomas ficou bem parecido, em torno de 85%, o que é muito bom para um modelo de Reconhecimento de Entidade Nomeada.

### Análise de Erro

Por fim, é sempre importante fazer uma análise de erro para entender as fraquezas e forças do seu modelo, entendendo potenciais melhorias ou até vendo possíveis viéses.
<br>Podemos fazer isso extraindo a loss e a classe prevista para cada token de cada registro, comparando com a classe verdadeira do token do registro. Na análise de erro, podemos olhar para os registros com as maiores loss.

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    batch = data_collator(features)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    with torch.no_grad():
        output = trainer.model(input_ids, attention_mask)
        predicted_label = torch.argmax(output.logits, axis=-1).cpu().numpy()
    loss = cross_entropy(output.logits.view(-1,7), labels.view(-1), reduction="none")
    loss = loss.view(len(input_ids), -1).cpu().numpy()
    return {"loss": loss, "predicted_label": predicted_label}

In [ ]:
valid_set = panx_en_encoded["validation"]
valid_set = valid_set.map(forward_pass_with_label, batched=True, batch_size=32)
df = valid_set.to_pandas()

Precisamos extrair is tokens, até então só temos os ids para melhorar nossa leitura. Além disso, vamos retirar o padding adicionado anteriormente limitando pelo tamanho do dado de entrada (input_ids).

In [ ]:
index2tag[-100] = "IGN"
df["input_tokens"] = df["input_ids"].apply(lambda x: tokenizer.convert_ids_to_tokens(x))
for label in ["predicted_label", "labels"]:
    df[label] = df[label].apply(lambda x: [index2tag[i] for i in x])
for col in ["loss", "predicted_label"]:
    df[col] = df.apply(lambda x: x[col][:len(x["input_ids"])], axis=1)
df.head(1)

Agora vamos olhar para os tokens de forma individual, explodindo as listas, criando uma linha por token. Também vamos retirar os labels IGN que possuem loss zero de qualquer forma.

In [ ]:
df_tokens = df.apply(pd.Series.explode)
df_tokens = df_tokens[df_tokens["labels"] != "IGN"]
df_tokens["loss"] = df_tokens["loss"].astype(float).round(2)
df_tokens.head()

Agora podemos agrupar os dados por token e agregar a loss, extraindo contagem, média e soma. Podemos ver que o token que acumulou a maior loss no dataset de validação é o _, seguido por _of e assim por diante...
<br>Isso nos dá alguns insights importantes:
- o token `_` representa espaço, ele ter obtido a maior loss acumulada não é surpresa, visto que ele é o token mais frequente também. Além disso, a média da loss desse token indica que o modelo não tem problemas em classificar ele.
- os tokens `_of`, `_the`, `_and` e outros na lista aparencem com bastante frequência. Esses tokens aparecem bastante juntos com entidade nomeadas e as vezes fazem parte delas, o que explica o porque o modelo fica confuso com eles.
- Parênteses no início das palavras apareceram com uma frequencia relativamente alta e uma média de loss bem alta também, precisando de investigação.

In [ ]:
df_tokens.groupby("input_tokens")[["loss"]].agg(
    ["count", "mean", "sum"]
).droplevel(level=0, axis=1).sort_values(
    by="sum", ascending=False
).reset_index().round(2).head(10).T

Analisando pelos labels, podemos ver que a maior loss acumulada é da classe I-ORG, sendo onde nosso modelo tem maior dificuldade.

In [ ]:
df_tokens.groupby("labels")[["loss"]].agg(
    ["count", "mean", "sum"]
).droplevel(level=0, axis=1).sort_values(
    by="sum", ascending=False
).reset_index().round(2).head(10).T

Podemos quebrar ainda mais e ver a matriz de confusão. Abaixo vemos que:
- Nosso modelo se confunde entre o B-ORG (início de organização) e o I-ORG (token subsequente).
- O modelo também se confunde bastante entre a classe B-PER com O e I-PER.
- Em geral nosso modelo é um bom modelo, podemos ver isso claremente pela diagonal da matriz de confusão.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    _, ax = plt.subplots(figsize=(6,6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Matriz de confusão normalizada")
    plt.show()
plot_confusion_matrix(df_tokens["labels"], df_tokens["predicted_label"], tags.names)

Agora analisamos a nível de token, vamos avaliar alguns exemplos (sequências). Para isso, voltamos para nosso dataset não explodido e somamos as losses de cada token na sequência.

In [ ]:
def get_samples(df):
    for _, row in df.iterrows():
        labels, preds, tokens, losses = [], [], [], []
        for i, _ in enumerate(row["attention_mask"]):
            if i not in {0, len(row["attention_mask"])}:
                labels.append(row["labels"][i])
                preds.append(row["predicted_label"][i])
                tokens.append(row["input_tokens"][i])
                losses.append(f"{row['loss'][i]:.2f}")
        df_tmp = pd.DataFrame({"tokens": tokens, "labels": labels, "preds": preds, "losses": losses}).T
        yield df_tmp

Abaixo podemos ver os 2 registros com maior loss total. Algumas coisas interessantes nesses registros:
- Temos um token `_alt` classificado `B-ORG`, mas é uma classificação errada, assim como `Justin Timberlake` não é uma organização, nosso modelo na verdade está correto. Isso pode acontecer, visto que os dados do PAN-X foram gerados em um process automático. Essas classificações são consideradas "padrão prata", contrastando com as classificações "padrão ouro" feitas por humanos. Mesmo com classificações "padrão ouro", ainda podemos ter erros nos labels dos nossos dados. 

In [ ]:
df["total_loss"] = df["loss"].apply(sum)
df_tmp = df.sort_values(by="total_loss", ascending=False).head(2)
for sample in get_samples(df_tmp):
    display(sample)

Conforme vimos anteriormente, parênteses tinham uma loss relativamente alta, vamos ver alguns exemplos?
<br> Em geral os parênteses e seus conteúdos não fazem parte das entidades nomeadas, mas em alguns casos parece que isso está acontecendo.

In [ ]:
df_tmp = df.loc[
    df["input_tokens"].apply(lambda x: u"\u2581(" in x)
].sort_values(by="total_loss", ascending=False).head(2)
for sample in get_samples(df_tmp):
    display(sample)

Com algumas análises simples, já conseguimos identificar fraquezas e forças dentro do nosso modelo e dos nossos dados. Agora podemos iterar sobre ele para limpar nossos labels, nossos dados, fazer otimização de hiperparâmetro como fizemos no modelo de análise de sentimento da aula passada, reanalisar os errors, e ir cada vez melhorando mais nosso modelo até estarmos satisfeitos com seus performance.

# O que você viu nessa aula?

Nessa aula, vimos o que é e como treinar um NER, que é uma tarefa muito comum em classificação de textos. Usamos o BERT para fine-tuning de modelos em português e conseguimos bons resultados